<a href="https://colab.research.google.com/github/SprihaT/XOR-Logic-Gate/blob/main/Extended_Backpropogation_XOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import numpy as np

class Neuron():
    def __init__(self, position_in_layer, is_output_neuron=False):
        self.weights = []
        self.inputs = []
        self.output = None
        self.updated_weights = []
        self.is_output_neuron = is_output_neuron
        self.delta = None
        self.position_in_layer = position_in_layer

    def attach_to_output(self, neurons):
        self.output_neurons = neurons

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Slope of the sigmoid function: used to calculate the change in neuron activation and weights
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def init_weights(self, num_input):
        limit = np.sqrt(6 / (num_input + 1))  # Xavier initialization to keep weights stable (prevents exploding or vanishing weights)
        self.weights = [random.uniform(-limit, limit) for _ in range(num_input + 1)]

    def predict(self, row):
        self.inputs = row + [1]  # Add bias term
        activation = sum(w*x for w, x in zip(self.weights, self.inputs))
        self.output = self.sigmoid(activation)
        return self.output

    def update_neuron(self):
        self.weights = self.updated_weights[:]

    def calculate_update(self, learning_rate, target):
        if self.is_output_neuron:
            self.delta = (target - self.output) * self.sigmoid_derivative(self.output)
        else:
            cur_weight_index = self.position_in_layer
            self.delta = sum(out.delta * out.weights[cur_weight_index] for out in self.output_neurons) * self.sigmoid_derivative(self.output)

        self.updated_weights = [w + learning_rate * self.delta * inp for w, inp in zip(self.weights, self.inputs)]

class Layer():
    def __init__(self, num_neuron, is_output_layer=False):
        self.is_output_layer = is_output_layer
        self.neurons = [Neuron(i, is_output_layer) for i in range(num_neuron)]

    def attach(self, layer):
        for neuron in self.neurons:
            neuron.attach_to_output(layer.neurons)

    def init_layer(self, num_input):
        for neuron in self.neurons:
            neuron.init_weights(num_input)

    def predict(self, row):
        return [neuron.predict(row) for neuron in self.neurons]

class MultiLayerPerceptron():
    def __init__(self, learning_rate=0.05, num_iteration=100000):
        self.layers = []
        self.learning_rate = learning_rate
        self.num_iteration = num_iteration

    def add_output_layer(self, num_neuron):
        self.layers.insert(0, Layer(num_neuron, is_output_layer=True))

    def add_hidden_layer(self, num_neuron):
        hidden_layer = Layer(num_neuron)
        hidden_layer.attach(self.layers[0])
        self.layers.insert(0, hidden_layer)

    def update_layers(self, target):
        for layer in reversed(self.layers):
            for neuron in layer.neurons:
                neuron.calculate_update(self.learning_rate, target)

        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.update_neuron()

    def fit(self, X, y):
        num_feature = len(X[0])

        self.layers[0].init_layer(num_feature)
        for i in range(1, len(self.layers)):
            self.layers[i].init_layer(len(self.layers[i - 1].neurons))

        for i in range(self.num_iteration):
            for row, target in zip(X, y):
                self.predict(row)
                self.update_layers(target)

            if i % 1000 == 0:
                # Mean Squared Error Cost function
                total_error = sum((target - self.predict(row)) ** 2 for row, target in zip(X, y)) / len(X)
                print(f"Iteration {i} with error = {total_error:.6f}")

    def predict(self, row):
        activations = row
        for layer in self.layers:
            activations = layer.predict(activations)
        return activations[0]
# BinaryXOR Original Training Data
X = [[0, 0], [0, 1], [1, 0], [1, 1]]
y = [0, 1, 1, 0]

# Extended Training Data
X.extend([
    [0.5, 0], [0.5, 0.25], [0.5, 0.5], [0.5, 0.75], [0.5, 1],
    [0.25, 0], [0.25, 0.25], [0.25, 0.5], [0.25, 0.75], [0.25, 1],
    [0.75, 0], [0.75, 0.25], [0.75, 0.5], [0.75, 0.75], [0.75, 1],
    [0.1, 0.1], [0.2, 0.2], [0.3, 0.3], [0.4, 0.4], [0.5, 0.5], [0.6, 0.6], [0.7, 0.7], [0.8, 0.8], [0.9, 0.9]
])
y.extend([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

clf = MultiLayerPerceptron(learning_rate=0.05, num_iteration=10000)
clf.add_output_layer(num_neuron=1)
clf.add_hidden_layer(num_neuron=6)
clf.add_hidden_layer(num_neuron=4)
clf.fit(X, y)

print("Testing Results:")
print("Expected 0.0, got:", clf.predict([0, 0]))
print("Expected 1.0, got:", clf.predict([0, 1]))
print("Expected 1.0, got:", clf.predict([1, 0]))
print("Expected 0.0, got:", clf.predict([1, 1]))

Iteration 0 with error = 0.256813
Iteration 1000 with error = 0.249660
Iteration 2000 with error = 0.246022
Iteration 3000 with error = 0.131194
Iteration 4000 with error = 0.009649
Iteration 5000 with error = 0.002208
Iteration 6000 with error = 0.001075
Iteration 7000 with error = 0.000677
Iteration 8000 with error = 0.000482
Iteration 9000 with error = 0.000370
Testing Results:
Expected 0.0, got: 0.018373049822515718
Expected 1.0, got: 0.9999889000671645
Expected 1.0, got: 0.9998607507473661
Expected 0.0, got: 0.01808724778685665


In [ ]:
# prompt: on a 3D plane, graph the trained clf object for the inputs x and y (both between the range of 0 to 1) and the resulting output z. use plotly.graph_objects as go instead of matplotlib.pyplot as plt, because I want the graph to be rotatable

import plotly.graph_objects as go
import numpy as np

# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(z=np.array([[clf.predict([x,y]) for x in np.linspace(0,1,100)] for y in np.linspace(0,1,100)]), x=np.linspace(0,1,100),y=np.linspace(0,1,100))])
fig.update_layout(title='Trained XOR Output', autosize=False,width=800,height=800,margin=dict(l=65, r=50, b=65, t=90))
fig.show()